# Purpose
from https://www.tv2kosmopol.dk/valg/stemmeseddel
to a .csv file

target municipalities:
- København
- Frederiksberg
- Hvidovre
- Tårnby
- Dragør
- Rødovre
- Brøndby
- Gentofte
- Gladsaxe
- Herlev

# requirements
- beautiful soup 4
- pandas
- lxml

In [56]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import csv
from enum import Enum
import re

root_url = "https://www.tv2kosmopol.dk/valg/stemmeseddel"
cand_url_start = "https://www.tv2kosmopol.dk/kandidat/"
csv_filename = 'data.csv'
regex_pattern = re.compile(r'/kandidat/\d+')

#municipalities = ["København", "Frederiksberg", "Hvidovre", "Tårnby", "Dragør", "Rødovre", "Brøndby", "Gentofte", "Gladsaxe", "Herlev"]
test_mun = ["frederiksberg"]
topics = ["Beredskab og sikkerhed", #order may differ
        "Erhverv / administration", 
        "x Kommune", # todo: insert kommune
        "Kultur, idræt og fritid",
        "Miljø og klima",
        "Skat",
        "Skole / dagtilbud for børn",
        "Social- og integrationsområdet",
        "Sundhed",
        "Trafik",
        "Ældre"]

scale = {
    "Helt uenig" : 0,
    "Overvejende uenig" : 1,
    "Hverken/eller" : 2,
    "Overvejende enig" : 3,
    "Helt enig" : 4
}

info_labels = ["municipality","id", "name", "votes", "elected", "age", "party"]

In [ ]:
def get_html(url):
    document = requests.get(url)
    return document.text # JSON


def get_id(tag):
    num = tag['data-id']
    return int(num)

def get_name(tag):
    p = tag.find('p', class_='sc-font-bold')
    return p.get_text() if p else ""

def votes_elected(tag):
    p = tag.find_all('p', class_='sc-font-light')
    votes = p[0].get_text().replace(" stemmer", "")
    votes = votes.replace(".", "")

    elected = p[1].get_text()
    elected = elected == "Valgt: Ja"
    return int(votes), elected


def get_surface_info(html):
    # read without going into personal site
    id = get_id(html)
    name = get_name(html)
    votes, elected = votes_elected(html)
    return [id,name,votes,elected]

def get_age(soup):
    s = soup.find('h1', class_="sc-text-3xl sc-font-semibold sc-text-gray-900 sc-mb-2")#.get_text()
    if s is None : return -1
    s = s.get_text()
    if not "år" in s : return -1
    target = ", "
    parts = s.split(target, maxsplit = 1)
    age = parts[1].replace(" år", "")
    return int(age)

def get_party_letter(soup):
    letter = soup.find('p', class_="sc-aspect-square sc-w-8 sc-h-8 sc-flex sc-items-center sc-justify-center")
    if letter is None : return ""
    return letter.get_text(strip=True)

def get_candidate_info(html):
    info = get_surface_info(html)
    id = info[0]
    
    # go to personal site
    personal_html = get_html(cand_url_start + str(id))
    personal_soup = bs(personal_html, 'lxml')
    
    # handle case with no info given

    age = get_age(personal_soup)
    party = get_party_letter(personal_soup)

    # relationship status
    # num kids
    # area
    # job
    # test and answers
    # short text
    # longer text

    # return string with info
    return info + [age,party]

In [58]:
# main loop
# csv: https://www.geeksforgeeks.org/python/how-to-create-a-csv-file-using-python/
with open(csv_filename, mode='w', newline='\n') as file:
    writer = csv.writer(file)
    
    writer.writerow(info_labels)

    for municipality in test_mun:
        url = root_url + "-" + municipality
        html = get_html(url) # go to specific municipality
        soup = bs(html, 'lxml') # using lxml parser which has C dependency and is very fast

        # for every candidate
        candidates = soup.find_all('a', class_="sc-relative sc-flex sc-items-center sc-gap-4 hover:sc-scale-[1.02] sc-transition-all sc-duration-200 sc-ease-in-out")#, href=regex_pattern)#
        for cand in candidates:
            info = get_candidate_info(cand)
            info.insert(0,municipality)
            writer.writerow(info)

# close file again?